In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import current_timestamp

In [0]:
%run "../includes/configuration"

In [0]:
laptimes_schema  = StructType(fields = [
    StructField("raceId", IntegerType(), False),
    StructField("driverId", IntegerType(), True),
    StructField("lap", IntegerType(), True),
    StructField("position", IntegerType(), True),
    StructField("time", StringType(), True),
    StructField("milliseconds", IntegerType(), True)
])

In [0]:
df_laptimes = spark.read \
    .option("header", True) \
    .schema(laptimes_schema) \
    .csv(f"{raw_folder_path}/lap_times/*.csv")

In [0]:
df_laptimes_renamed = df_laptimes.withColumnRenamed("driverId", "driver_id") \
    .withColumnRenamed("raceId", "race_id") \
    .withColumn("ingestion_date", current_timestamp())


In [0]:
df_laptimes_renamed.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.lap_times")